# Lab 1: Introduction to OpenMP

The objective of this lab is to get familiar with the basic concepts behind OpenMP. Some of these concepts are shared with other programming models, and are important to understand how systems are programmed in parallel. These concepts are introduced directly using OpenMP syntax. It is not expected for the reader to know OpenMP, but they should be familiar with C-like syntax.

This tutorial is expected to run in a unix-like environment.

## Table of content:

* Thread and multithread
    * First parallel program
    * Thinking in parallel
    * Exercise 1
* Memory: Shared, private, distributed
    * Atomic operations
    * Private vs Firstprivate
    * Reductions
    * Lastprivate
    * Exercise 2
* OpenMP Syntax
* Function outlining, implementation and runtime


## Thread and multithreads

Simply speaking, a thread is a worker that execute instructions. Current CPU architectures are mostly multi threaded, where each worker is independent to each other. 

Let's see how many cores are in our current system using `lscpu`.

In [1]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         48 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  12
  On-line CPU(s) list:   0-11
Vendor ID:               AuthenticAMD
  Model name:            AMD Ryzen 5 5600G with Radeon Graphics
    CPU family:          25
    Model:               80
    Thread(s) per core:  2
    Core(s) per socket:  6
    Socket(s):           1
    Stepping:            0
    CPU max MHz:         4464.0000
    CPU min MHz:         400.0000
    BogoMIPS:            7800.46
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mc
                         a cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall n
                         x mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_go
                         od nopl nonstop_tsc cpuid extd_apicid aperfmperf rapl p
                         ni pclmulqdq monitor ssse3 fma cx16 sse4_1 sse4_2 x2api

In the output above you can find the number of CPUs (e.g. 8) and the number of Threads per CPU (e.g. 2). Each Thread is capable of independently handle a different stream of instructions. However, **software often creates more threads than available in the hardware system**. 

You can ignore the syntax in the following command, but its output will show the number of threads running within all processes in the system (or at least those that your user can obtain information about). __Most likely you will find that the number of *software* threads is much larger than the number of *hardware* threads. This is because the operating system uses a scheduling scheme to execute all the threads concurrently__. 

The focus of this lab is not to learn about OS threads, but it is worth knowing that the number of *software* threads in a given program can be larger than the number of *hardware* threads running in the system.

In [90]:
!ps -eo nlwp | tail -n +2 | awk '{ num_threads += $1 } END { print num_threads }'

22


### Let's create our first threaded program

```C
#include <omp.h>

int main() {
    #pragma omp parallel num_threads(10)
    {
        printf("Hello from thread %d\n",omp_get_thread_num());
    }
    return 0;
}
```

In [11]:
# First, let's compile it
!gcc -fopenmp C/parallel.c -o C/parallel.exe

In [12]:
# Now it is time to run it.
!C/./parallel.exe

Hello from thread 3
Hello from thread 10
Hello from thread 1
Hello from thread 0
Hello from thread 9
Hello from thread 7
Hello from thread 2
Hello from thread 6
Hello from thread 11
Hello from thread 5
Hello from thread 8
Hello from thread 4


You should immediately notice that (likely) the output of the above program is not in a given order. This is because all the threads are running concurrently, and, if the number of hardware threads is larger than 1, a set of them may be running in parallel.

### Do it yourself

Open the file [parallel.c](C/parallel.c) and play with different number of threads by changing the value inside the clause `num_threads()`, and re-running the above two commands.

### Thinking in parallel

If you're experienced in sequential programming, __you're likely familiar with writing code for a single thread.  When thinking sequentially, the developer must primarly think of the instructions that are executed. Parallel programming adds an additional complexity to the development program. When writting programs, a developer must think of the following aspects of the code__:

* **Workers creation**: How to create workers and how many to create
* **Work assignment**: How to assign work to different workers
* **Workers/resources communication and coordination**: How workers communicate and synchronize in order to coordinate their work.

```
Note: Different programming models exist that balance these three tasks. Here I am referring to the most popular programming models. Allow me to be simplistic here.
```

#### The fork-join model - An example.

OpenMP is mainly known for its *Fork-Join* model. Programs execute with an initial sequential thread until a directive (code anotation) is reached that initiates parallel threads (workers). Different directives are used to assign work to these threads. At the end of the parallel region, threads synchronize, forming again a single thread.

For the above example. **worker creation** is achieved through the `#pragma omp parallel num_threads(10)` directive. It allows to create 10 threads. Each worker is associated with an identifier, obtained through the `omp_get_thread_num()` function call. **Work assignment** is achieved through the code in the region enclosed by the brackets `{...}`. All threads are executing the same set of instructions, in this case only `printf("hello from thread %d\n",...);`. Notice that having a different identifier allows for each thread to access different data, or follow different paths. Finally **worker synchronization and coordination** is relatively simple in this example, since there's not much communication between them. However, at the end of the parallel region, all workers must wait for each other to finish before continuing with the sequential code. 

### Exercise 1

Can you modify the code above to make the odd and even threads print something different?Go to exercise1.c and use the code region below to build and execute your code

In [4]:
!gcc -fopenmp Exercises/exercise1.c -o Exercises/exercise1.exe && Exercises/./exercise1.exe

This thread is odd 1
This thread is even 8
This thread is even 0
This thread is odd 9
This thread is even 2
This thread is odd 7
This thread is even 6
This thread is odd 3
This thread is even 10
This thread is odd 5
This thread is even 4
This thread is odd 11


## Memory: Shared vs Private

When programming we often imagine memory as a single "monolithic" element. However, this is not the reality. __Current architectures feature a complex memory organization that includes registers, caches, multiple DRAM banks__, and even devices with a different memory space. When programming in parallel, such complex structures become more important for correctness and performance.

__Multithreading programming often features shared memory, meaning that all threads have access to the same varibles__, located in the same memory address (I expect the reader to be familiar with pointers). __However, it is also possible for each thread to have private memory, even if the name of variables are the same__.

Take for example the next program:

```C
int main() {
    int i;
    double share;
    int Array[10];

    printf("Address of i prior to the parallel region is: %lx\n",(unsigned long)&i);
    printf("Address of shared prior to the parallel region is: %lx\n",(unsigned long)&share);
    printf("Address of Array prior to the parallel region is: %lx\n",(unsigned long)Array);

    #pragma omp parallel private(i, Array) shared(share)
    {
        printf("Address of i as seen by thread %d: %lx\n", omp_get_thread_num(), (unsigned long)&i);
        printf("Address of Array as seen by thread %d: %lx\n", omp_get_thread_num(), (unsigned long)Array);
        printf("Address of share as seen by thread %d: %lx\n", omp_get_thread_num(), (unsigned long)&share);
    }
}
```

In [28]:
!gcc -fopenmp C/memory.c -o C/memory.exe

In [16]:
!C/./memory.exe

Address of i prior to the parallel region is: 7ffed175defc
Address of shared prior to the parallel region is: 7ffed175df00
Address of Array prior to the parallel region is: 7ffed175df10
Address of i as seen by thread 6: 7e6a7ff78d4c
Address of Array as seen by thread 6: 7e6a7ff78d50
Address of share as seen by thread 6: 7ffed175df00
Address of i as seen by thread 5: 7e6a80779d4c
Address of Array as seen by thread 5: 7e6a80779d50
Address of share as seen by thread 5: 7ffed175df00
Address of i as seen by thread 9: 7e6a7e775d4c
Address of Array as seen by thread 9: 7e6a7e775d50
Address of share as seen by thread 9: 7ffed175df00
Address of i as seen by thread 7: 7e6a7f777d4c
Address of Array as seen by thread 7: 7e6a7f777d50
Address of share as seen by thread 7: 7ffed175df00
Address of i as seen by thread 1: 7e6a8277dd4c
Address of Array as seen by thread 1: 7e6a8277dd50
Address of share as seen by thread 1: 7ffed175df00
Address of i as seen by thread 2: 7e6a81f7cd4c
Address of Array as se

You can play with this code going to [memory.c](C/memory.c)

__Private memory allows for variables to be visible only by the current worker. Shared memory, allows variables to be visible from and modified by multiple workers (read and write)__. However, it is important to be careful. As previously mentioned, memory organization is complex, and requires additional coordination.

Take for example the following program:

```C
int main() {
    int i = 0;

    #pragma omp parallel shared(i) num_threads(1000)
    {
        i++;
    }
    
    printf("i = %d\n",i);
    return 0;
}
```

In [30]:
!gcc -fopenmp C/datarace.c -o C/datarace.exe

In [11]:
!C/./datarace.exe

i = 991


If 10000 workers are created, and each is adding 1 to the value of i, the "expected" value would be 10000. If you're running this in a machine that has multiple threads, I expect you to see a number less than 10000. Moreover, multiple executions may lead to different results. This is what is known as a **data race**. It occurs because reading i, incrementing, and writting to i are three different instructions. Therefore, it is possible for two threads to write the same value of i, increment it and obtain the same value twice.

For this reason, __shared memory requires additional coordination between workers__, such that reads and writes to the same region are perceived in the expected order.

Notice that this is part of thinking about **workers/resource communication and coordination**. We are coordinating memory acesses and operations to variables, as they are shared across different workers.

You can modify the above code going to [datarace.c](C/datarace.c)

```
Note: Data races are a difficult aspect of parallel programming. Multiple executions may lead to different results, but among the different results that can be obtained, it is still possible to obtain the "expected" result. Some data races are more difficult to debug than others, because they have a higher chance of choosing the "expected" result. Imagine that 1 out of a million executions of your code in a given hardware shows a datarace. Now imagine this program running the automatic pilot of an airplane...
```

### Atomic operations

__Atomic operations allows different threads to perform different operations to the same memory location in a single instruction (or as if they were executed in a single instruction)__. In the example above, atomic operations would allow for read, increment and write to happen in a single instruction (or "atomically"). Thus, atomic operations can solve the datarace issue of the previous example.

```C
int main() {
    int i = 0;

    #pragma omp parallel shared(i) num_threads(1000)
    {
        #pragma omp atomic
        i++;
    }
    
    printf("i = %d\n",i);
    return 0;
}
```

In [8]:
!gcc -fopenmp C/atomic.c -o C/atomic.exe

In [12]:
!C/./atomic.exe

i = 1000


You can play with the above code going to [atomic.c](C/atomic.c)

### Private vs Firstprivate

Privatization of variables means that a single variable name will have different memory locations. Privatization does not guarantee that the new address has the same value of the original address (i.e. the address before the parallel region). 

Take for example the following code:

```C
int main() {
    int i = 999;

    printf("i is %d before parallel region\n",i);

    #pragma omp parallel private(i) num_threads(10)
    {
        printf("Thread %d sees %d on memory %lx\n", omp_get_thread_num(), i, (unsigned long)&i);
    }

    return 0;
}
```

In [34]:
!gcc -fopenmp C/private.c -o C/private.exe

In [35]:
!C/./private.exe

i is 999 before parallel region
Thread 0 sees 0 on memory 7ffeeca405a4
Thread 6 sees 0 on memory 7610bff3ad74
Thread 5 sees 0 on memory 7610c073bd74
Thread 8 sees 0 on memory 7610bef38d74
Thread 3 sees 0 on memory 7610c173dd74
Thread 1 sees 0 on memory 7610c273fd74
Thread 2 sees 0 on memory 7610c1f3ed74
Thread 4 sees 0 on memory 7610c0f3cd74
Thread 7 sees 0 on memory 7610bf739d74
Thread 9 sees 0 on memory 7610be737d74


You can play with the above code going to [private.c](C/private.c)

Firstprivate allows for each new address location to be initialized with the value prior to the parallel region. Take for example the following code.

```C
int main() {
    int i[3] = {999,888,666};

    printf("i is [%d,%d,%d] before parallel region\n",i[0],i[1],i[2]);

    #pragma omp parallel firstprivate(i) num_threads(10)
    {
        printf("Thread %d sees [%d,%d,%d] on memory %lx\n", omp_get_thread_num(), i[0],i[1],i[2], (unsigned long)i);
    }

    return 0;
}
```

In [36]:
!gcc -fopenmp C/firstprivate.c -o C/firstprivate.exe

In [37]:
!C/./firstprivate.exe

i is [999,888,666] before parallel region
Thread 6 sees [999,888,666] on memory 749aff73dd5c
Thread 0 sees [999,888,666] on memory 7ffe6d0e25ac
Thread 3 sees [999,888,666] on memory 749b00f40d5c
Thread 2 sees [999,888,666] on memory 749b01741d5c
Thread 7 sees [999,888,666] on memory 749afef3cd5c
Thread 1 sees [999,888,666] on memory 749b01f42d5c
Thread 8 sees [999,888,666] on memory 749afe73bd5c
Thread 4 sees [999,888,666] on memory 749b0073fd5c
Thread 9 sees [999,888,666] on memory 749afdf3ad5c
Thread 5 sees [999,888,666] on memory 749afff3ed5c


You can play with the above code going to [firstprivate.c](C/firstprivate.c)

Notice how each location for i is different on each thread, yet, all have the same expected value. __This also means that memory copies need to be performed in order to achieve this behavior. Hence, if your array is large, it may incur in additional overhead__.

There are still a lot of good reasons to privatize variables. 

### Reductions

So far we have discussed what happens to variables when going from a sequential region to a parallel region. However, what happens to these multiple private memory locations when a parallel region is over? In principle, private variables are discarded (freed) at the end of a parallel region, hence, if their value are important, it is necessary to store them into a shared location that lives after the parallel region. Yet, it is often desireable to update the original memory location of the variable.

But wait, all of these variables may have different values. How do I decide what's the final value to be used after the parallel region? Reductions are collective operations that aggregate the different values into a single value, by applying an operation. Ideally, this operation should be commutative, otherwise, how do I decide the order in which they are applied?

```
Note: Surprisingly enough OpenMP used to support the minus (-) operation for reductions. It wasn't until version 5.2 that they removed support for this operation. If thread 1 and thread 2 are reducing A, should it be At1 - At2 or At2 - At1?
```

Let's take for example the following code:

```C
int main() {
    int i = 99;

    printf("Value if i prior to parallel region is %d\n",i);

    // Private values are not transferred back
    #pragma omp parallel private(i)
    {
        i=1000;
    }
    printf("Value if i after parallel region with private(i) is %d\n",i);

    i = 0;
    // Reductions for addition.
    #pragma omp parallel reduction(+:i) num_threads(10)
    {
        i=1;
    }
    printf("Value if i after parallel region with reduction(+:i) is %d\n",i);

    // Reductions for max.
    #pragma omp parallel reduction(max:i) num_threads(20)
    {
        i=omp_get_thread_num();
    }
    printf("Value if i after parallel region with reduction(max:i) is %d\n",i);

    return 0;
}
```

In [38]:
!gcc -fopenmp C/reductions.c -o C/reductions.exe

In [39]:
!C/./reductions.exe

Value if i prior to parallel region is 99
Value if i after parallel region with private(i) is 99
Value if i after parallel region with reduction(+:i) is 10
Value if i after parallel region with reduction(max:i) is 19


You can play with the above code going to [reductions.c](C/reductions.c). Other reduction operations are:
* Multiplication (*)
* Minimun (min)
* Bitwise AND (&)
* Bitwise OR (|)
* Bitwise XOR (^)
* Logic AND (&&)
* Logic OR (&&)

### Lastprivate

Finally, there is lastprivate. Later on we will discuss more about loops and how to distribute them across workers. However, lastprivate allows for the value to be the last value in the iteration space. This means, if we have 10 iterations in a for loop, the value for i = 9 will be copied over.

This code shows this behavior. Let us ignore for now the `for` construct. We will go back to it later on.

```C
    int Array[10];
    int i, b;

    for (i = 0; i < 10; i++) {
        Array[i] = i;
    }

    #pragma omp parallel for lastprivate(b)
    for (i = 0; i < 10; i++)
    {
        b = Array[i];
    }
    printf("b is %d after the parallel region\n", b);

    return 0;
```

In [40]:
!gcc -fopenmp C/lastprivate.c -o C/lastprivate.exe

In [13]:
!C/./lastprivate.exe

b is 9 after the parallel region


You can play with this code going to [lastprivate.c](C/lastprivate.c).

### Exercise 2

Create a program that:
1. Initializes an array of 100 elements to random numbers by assigning a thread per element of the array. 
2. Finds the max value of the array. 
3. Finds the min value of the array.
4. Finds the average value of the array.

Go to [exercise2.c](Exercises/exercise2.c) and use the code region below to build and execute your code



In [89]:
!gcc -fopenmp Exercises/exercise2.c -o Exercises/exercise2.exe && Exercises/./exercise2.exe

 Thread 1: value 470 in index 9
 Thread 1: value 7695 in index 10
 Thread 1: value 8293 in index 11
 Thread 1: value 7321 in index 12
 Thread 1: value 4672 in index 13
 Thread 1: value 4251 in index 14
 Thread 1: value 9952 in index 15
 Thread 1: value 3353 in index 16
 Thread 1: value 702 in index 17
 Thread 7: value 312 in index 60
 Thread 7: value 8264 in index 61
 Thread 7: value 9794 in index 62
 Thread 5: value 8959 in index 44
 Thread 5: value 7327 in index 45
 Thread 5: value 5807 in index 46
 Thread 5: value 3834 in index 47
 Thread 5: value 4070 in index 48
 Thread 5: value 3769 in index 49
 Thread 6: value 6199 in index 52
 Thread 8: value 4141 in index 68
 Thread 8: value 5410 in index 69
 Thread 8: value 3904 in index 70
 Thread 8: value 1441 in index 71
 Thread 8: value 9447 in index 72
 Thread 8: value 7438 in index 73
 Thread 8: value 5306 in index 74
 Thread 8: value 1748 in index 75
 Thread 10: value 9571 in index 84
 Thread 10: value 4629 in index 85
 Thread 10: valu